# Derivadas superiores

Hasta ahora hemos visto que, usando diferenciación automática, podemos calcular la derivada de funciones de una variable esencialmente con un error del orden del epsilon de la máquina.

La pregunta que abordaremos ahora, es cómo hacer para calcular la segunda derivada, y derivadas de orden superior.

Una posibilidad, específica para el caso de la segunda derivada, es proceder como en el caso anterior, es decir, definir una *terna ordenada* donde la primer componente es el valor de la función en $x_0$, i.e., $f(x_0)$, el de la segunda es el valor de la primer derivada $f'(x_0)$, y la tercer componente tiene el valor de la segunda derivada, $f^{(2)}(x_0) = f''(x_0)$. 


Con esta definición, las operaciones aritméticas vienen dadas por:

\begin{eqnarray}
\vec{u} + \vec{v} & = & (u + v, \quad u'+ v', \quad u''+v''),\\
\vec{u} - \vec{v} & = & (u - v, \quad u'- v', \quad u''-v''),\\
\vec{u} \times \vec{v} & = & (u v, \quad u v' + u' v, \quad u v'' + 2 u' v' + u'' v),\\
\frac{\vec{u}}{\vec{v}} & = & \Big( \frac{u}{v}, \quad \frac{u'-( u/v) v'}{v}, \quad 
\frac{u'' - 2 (u/v)' v' - (u/v)v'' }{v}\Big).\\
\end{eqnarray}

Claramente, este proceso es muy ineficiente para derivadas de orden aún más alto, dado que las expresiones se complican y es fácil cometer errores.

# Series de Taylor

El punto importante a recordar, es que las derivadas de orden superior de una función $f(x)$ en un punto $x_0$ están contenidas en el desarrollo de Taylor de la función entorno a ese punto. La suposición importante en esto es que $f(x)$ es suficientemente suave; por simplicidad supondremos que $f(x)$ es ${\cal C}^\infty$ y que estamos suficientemente cerca del punto $x_0$, i.e., $|x-x_0|\ll 1$. 


La serie de Taylor de $f(x)$ viene dada por

\begin{eqnarray}
f(x) & = & f(x_0) + f^{(1)}(x_0) (x-x_0) + \frac{f^{(2)}(x_0)}{2!} (x-x_0)^2 + \dots + \frac{f^{(k)}(x_0)}{k!} (x-x_0)^k + \dots,\\
& = & f_{[0]} + f_{[1]} (x-x_0) + f_{[2]} (x-x_0)^2 + \dots + f_{[k]} (x-x_0)^k + \dots,\\
\end{eqnarray}

donde los coeficientes *normalizados* de Taylor $f_{[k]}$ que aparecen en la segunda línea de la ecuación anterior se definen como

\begin{equation}
f_{[k]} = \frac{f^{(k)}(x_0)}{k!} = \frac{1}{k!}\frac{{\rm d}^k f}{{\rm d}x^k}(x_0).
\end{equation}



Vale la pena **enfatizar** que la expresión anterior es exacta en tanto que la serie **no** sea truncada. En el caso de que la serie truncada a orden k, el [teorema de Taylor](https://en.wikipedia.org/wiki/Taylor%27s_theorem) asegura que el residuo (error del truncamiento) se puede escribir como:

\begin{equation}
{\cal R_{k}} = \frac{f^{(k+1)}\,(\xi)}{(k+1)!} (x-x_0)^{k+1},
\end{equation}

donde $\xi$ es un punto que pertenece al interval $[x_0,x]$.


Si la serie es truncada, la aproximación es un polinomio de orden $k$ (grado máximo es $k$) en $x$. Dado que los polinomios en una variable están definidos por $k+1$ coeficientes, entonces pueden ser mapeados a vectores en $\mathbb{R}^{k+1}$. 

Las operaciones aritméticas, en este caso, vienen dadas por:

\begin{eqnarray}
(f+g)_{[k]} & = & f_{[k]} + g_{[k]} ,\\
(f-g)_{[k]} & = & f_{[k]} - g_{[k]} ,\\
(f \cdot g)_{[k]} & = & \sum_{i=0}^k f_{[i]} \,g_{[k-i]} \, ,\\
\Big(\frac{f}{g}\Big)_{[k]} & = & \frac{1}{g_{[0]}}
\Big( f_{[k]} - \sum_{i=0}^{k-1} \big(\frac{f}{g}\big)_{[i]} \, g_{[k-i]} \Big) . \\
\end{eqnarray}

### Ejercicio

Implementen una nueva estructura paramétrica (`type`) que defina el tipo `Taylor`, donde el parámetro debe ser un subtipo de `Number`. Definan métodos que implementen las operaciones aritméticas básicas (`+`, `-`, `*`, `/`) y la igualdad (`==`). Esto deberá ser incluido en un módulo.

Incluyan pruebas (en el archivo "runtests.jl") para cada uno de los métodos que implementen.


In [2]:
"""
Definición de polinomios de Taylor, consta de dos partes: 'co' es un Array{T<:Number} que contiene los 
coeficientes ordenados NO NULOS del polinomio y 'gr' es el grado del polinomio de tal forma que 
Taylor{Number}([a₀,a₁,a₂,..,a_n],m) representa el polinomio a₀ + a₁x + a₂x² + ··· + aⁿxⁿ + ... + 0xᵐ
...
"""
type Taylor{T<:Number}
    
    co :: Array{T,1}
    gr :: Int     
    
    function Taylor(co::Array{T,1}, gr::Int) #Para que los Taylor esten bien definidos
        if gr < length(co)-1
            error("""El grado del polinomio debe ser mayor o igual al numero de coeficientes menos uno. 
            Puede utilizar Taylor([a₀,a₁,...,aₙ]) sin especificar el grado para asignar automaticamente 
            el grado mínimo: Taylor([a₀,a₁,...,aₙ],n)""")
        end
        new(co,gr)
    end  
    
end

Taylor{T<:Number}(co::Array{T,1},gr::Int) = Taylor{T}(co,gr) #Para no tener que especificar el Tipo de los 
                                                             #coeficientes

Taylor{T}(co::Array{T,1}) = Taylor(co,length(co)-1) #Define un Taylor sin especificar el grado
    
Taylor(co::Number) = Taylor([co]) #El Taylor de una constante

import Base.promote

function promote(a::Taylor,b::Taylor)
    gr = maximum([a.gr,b.gr])
    A = zeros(typeof(promote(a.co)[1][1]), gr+1)
    B = zeros(typeof(promote(b.co)[1][1]), gr+1)
    for i in 1:length(a.co)
        A[i] = a.co[i]
    end
    for i in 1:length(b.co)
        B[i] = b.co[i]
    end
    return Taylor(A,gr),Taylor(B,gr),gr
end

function promote(a::Taylor,b::Taylor,c::Int)
    # Esta función toma dos Taylor y un entero c, y regresa los mismos dos Taylor pero ahora ambos con c+1 coeficientes
    # y por tanto representan ahora polinomios de grado c
    (c < length(a.co)-1 || c < length(b.co)-1) && error("""El nuevo grado, c = $c, debe ser mayor que el grado mínimo 
                                                        de los Taylor ingresados""")
    A = Taylor(a.co,c)
    A,B,gr = promote(A,b)
    return A,B
end

promote (generic function with 7 methods)

In [3]:
?Taylor

search: 

Definición de polinomios de Taylor, consta de dos partes: 'co' es un Array{T<:Number} que contiene los  coeficientes ordenados NO NULOS del polinomio y 'gr' es el grado del polinomio de tal forma que  Taylor{Number}([a₀,a₁,a₂,..,aₙ],m) representa el polinomio a₀ + a₁x + a₂x² + ··· + aⁿxⁿ + ... + 0xᵐ ...


Taylor



In [4]:
A = [1,2,3,4,5]
B = [1,2,3]
C = [1,2,3,4,5,6,7];

In [5]:
using Base.Test

In [6]:
import Base: +, -, *, /, ==

In [7]:
function ==(a::Taylor,b::Taylor)
    a = Taylor(a)
    b = Taylor(b)
    return a.co == b.co
end

function +(a::Taylor,b::Taylor)
    a,b,c = promote(a,b)
    d = a.co + b.co
    Taylor(resize!(d,findlast(d)), c)
end

function -(a::Taylor,b::Taylor)
    a,b,c = promote(a,b)
    d = a.co - b.co
    Taylor(resize!(d,findlast(d)), c)
end

function *(a::Taylor,b::Taylor)
    c = length(a.co) + length(b.co)
    grad_f = a.gr + b.gr
    a,b = promote(a,b,c)
    suma(i) = sum([a.co[j]*b.co[i-j+1] for j in 1:i])
    d = [suma(i) for i in 1:c+1]
    d = Array{typeof(promote(d)[1][1])}(d) # Como el tipo original de d es Array{Any,1}, lo convertimos a un     
                                           # Array{T<:Number,1}
    return Taylor(resize!(d,findlast(d)), grad_f)
end

function /(a::Taylor, b::Taylor)
    length(a.co) < length(b.co) && error("El grado del numerador debe ser mayor al del denominador")
    c = length(b.co)-1
    suma(i) = sum([(a.co[j+1]/b.co[j+1])*b.co[i-j+2] for j in 1:i])
    d = unshift!([a.co[i] - suma(i) for i in 1:c]./b.co[1], a.co[1]/b.co[1])
    d = Array{typeof(promote(d)[1][1])}(d)
    return Taylor(resize!(d,findlast(d)), c)
end

/ (generic function with 49 methods)

In [8]:
Taylor(A) + Taylor(C)

Taylor{Int64}([2,4,6,8,10,6,7],6)

In [9]:
Taylor(A) - Taylor(B)

Taylor{Int64}([0,0,0,4,5],4)

In [10]:
Taylor(A) * Taylor(B)

Taylor{Int64}([1,4,10,16,22,22,15],6)

In [11]:
Taylor(A,8) / Taylor(B,7)

Taylor{Float64}([1.0,-1.0,-3.0],2)

$$\frac{1}{g_{[0]}} \Big( f_{[k]} - \sum_{i=0}^{k-1} \big(\frac{f}{g}\big)_{[i]} \, g_{[k-i]} \Big)$$

No sé si en verdad esta bien definida la división, sobre todo el termino $(\frac{f}{g})_{[i]}$ me causa ruido y lo termine interpretando como $\frac{f_{[i]}}{g_{[i]}}$.

In [12]:
# Muestren que su código funciona con tests adecuados; para los detalles ver 
# http://julia.readthedocs.org/en/release-0.4/stdlib/test/
using Base.Test


# Funciones de polinomios

El siguiente punto, es cómo definir funciones de polinomios. 

Como veremos aquí, esto se basará en plantear una ecuación diferencial apropiada, cuya solución es, precisamente, la expresión que estamos buscando. Este punto es *importante*, y muestra que hay una conexión importante con la solución de ecuaciones diferenciales.

Como ejemplo consideraremos la función

\begin{equation}
E(x) = \exp\big(g(x)\big),
\end{equation}

donde 

\begin{equation}
g(x) = \sum_{k=0}^\infty g_{[k]} (x-x_0)^k
\end{equation}

está escrita como una serie de Taylor (¡exacta!) alrededor de $x_0$. 


El primer punto, es que escribiremos a $E(x)$ como una serie de Taylor alrededor de $x_0$, es decir,

\begin{equation}
E(x) = \sum_{k=0}^\infty E_{[k]} (x-x_0)^k.
\end{equation}

El objetivo es determinar $E_{[k]}$ para *toda* $k$.

Dado que $E(x)$ esun polinomio en $x$, su derivada viene dada por

\begin{equation}
\frac{{\rm d} E(x)}{{\rm d}x} = \sum_{k=1}^\infty k E_{[k]}\, (x-x_0)^{k-1} .
\end{equation}

Por otra parte, la derivada de $E(x)$ en términos de $g(x)$ está dada por

\begin{equation}
\frac{{\rm d} E(x)}{{\rm d}x} = \exp\big(g(x)\big) \frac{{\rm d} g(x)}{{\rm d}x} = E(x) \frac{{\rm d} g(x)}{{\rm d}x},
\end{equation}

donde del lado derecho aparece la derivada de $g(x)$. Ya que $g(x)$ *también* está escrita en forma polinomial, su derivada es

\begin{equation}
\frac{{\rm d} g(x)}{{\rm d}x} = \sum_{k=1}^\infty k g_{[k]}\, (x-x_0)^{k-1} .
\end{equation}


Tenemos, entonces, todo lo que requerimos para escribir el lado derecho de la ecuación diferencial y explotar la aritmética de polinomios. 

\begin{eqnarray}
E(x) \frac{{\rm d} g(x)}{{\rm d}x}& = & 
\Big[ \sum_{k=0}^\infty E_{[k]} (x-x_0)^k \Big]
\Big[ \sum_{j=1}^\infty j g_{[j]} (x-x_0)^{j-1}\Big] \\
 & = & \sum_{k=1}^\infty \Big[ \sum_{j=0}^k j g_{[j]} E_{[k-j]} \; \Big] (x-x_0)^{k-1} .\\
\end{eqnarray}

La segunda línea se obtiene reordenando los términos al fijar la potencia de $(x-x_0)$, esto es, $k+j$ se toma como un nuevo índice ($k$), y el nuevo índice $j$ describe el índice del producto de los polinomios. (La potencia se deja de la forma $k-1$ ya que el lado izquierdo de la ecuación aparece así.)

Igualando con el lado izquierdo de la ecuación diferencial, que sólo involucra a la derivada de $E(x)$, tenemos que se debe cumplir

\begin{equation}
E_{[k]} = \frac{1}{k} \sum_{j=0}^k j g_{[j]} \, E_{[k-j]} = 
\frac{1}{k} \sum_{j=0}^{k} (k-j) g_{[k-j]} \, E_{[j]}, \qquad k=1,2,\dots,
\end{equation}

incluyendo *la condición inicial*

\begin{equation}
E_{[0]} = \exp\big(g(x_0)\big).
\end{equation}

Estas relaciones *de recurrencia* permiten calcular $\exp\big(g(x)\big)$, para cualquier polinomio $g(x)$.

Para el caso concreto $g(x) = x$ alrededor de $x_0=0$, donde tenemos $g_{[j]} = \delta_{j,1}$, obtenemos

\begin{eqnarray}
E_{[0]} & = & 1,\\
E_{[k]} & = & \frac{1}{k} E_{[k-1]} = \frac{1}{k(k-1)} E_{[k-2]} = \dots = \frac{1}{k!} E_{[0]} = \frac{1}{k!}\ ,
\end{eqnarray}

que es el resultado bien conocido.

### Ejercicio

Obtengan las relaciones de recurrencia para las funciones $L(x) = \log\big(g(x)\big)$, $P_\alpha(x) = \big(g(x)\big)^\alpha$, $S(x) = \sin\big(g(x)\big)$, $C(x) = \cos\big(g(x)\big)$ usando el mismo procedimiento que arriba. Implementen métodos adecuados para estas funciones en el módulo, actuando sobre estructuras `Taylor` e incluyan pruebas.

**NOTA** Los ejercicios de este notebook constituyen el contenido de la Tarea6.

### Solución 

### $L(x)$ 

$$ L(x) = log(g(x)) $$

$$\frac{dL(x)}{dx} = g^{-1}(x) \frac{dg(x)}{dx}$$

pero

$$g(x) = \sum^\infty_{k=0} g_{[k]} (x-x_0)^{k}$$

y

$$\frac{dg(x)}{dx} = \sum^\infty_{l=1} l\ g_{[l]} (x-x_0)^{l-1}$$

entonces

$$\frac{dL(x)}{dx} = \frac{\sum^\infty_{l=1} l\ g_{[l]} (x-x_0)^{l-1}}{\sum^\infty_{k=0} g_{[k]} (x-x_0)^{k}}$$

pero por otro lado

$$\frac{dL(x)}{dx} = \sum^\infty_{k=1} k\ L_{[k]} (x-x_0)^{k-1}$$

igualando las ultimas dos ecuaciones

$$\sum^\infty_{k=1} k\ L_{[k]} (x-x_0)^{k-1} = \frac{\sum^\infty_{l=1} l\ g_{[l]} (x-x_0)^{l-1}}{\sum^\infty_{k=0} g_{[k]} (x-x_0)^{k}}$$

multiplicando ambos lados por el denominador

$$\sum^\infty_{l=1} l\ g_{[l]} (x-x_0)^{l-1} = \Big(\sum^\infty_{k=1} k\ L_{[k]} (x-x_0)^{k-1}\Big) \Big(\sum^\infty_{l=0} g_{[l]} (x-x_0)^{l}\Big)$$

fijandonos unicamente en la multiplicación de las sumas, que es completamente análoga a la del ejemplo, tenemos la siguiente igualdad en general

$$\Big(\sum^\infty_{i=0} f_{[i]} (x-x_0)^{i}\Big) \Big(\sum^\infty_{j=1} j\ g_{[j]} (x-x_0)^{j-1}\Big) = \sum^\infty_{i=1} \Big[\sum^i_{j=0} j\ g_{[j]}\ f_{[i-j]}\Big] (x - x_0)^{i-1} $$

Entonces obtenemos

$$ \sum^\infty_{l=1} l\ g_{[l]} (x-x_0)^{l-1} = \sum^\infty_{l=1} \Big[\sum^l_{k=0} k\ L_{[k]}\ g_{[l-k]}\Big] (x-x_0)^{l-1}$$

por lo tanto si $l \neq 0$

$$l\ g_{[l]} = \sum^l_{k=0} k\ L_{[k]}\ g_{[l-k]}$$

$$l\ g_{[l]} = \sum^{l-1}_{k=0} k\ L_{[k]}\ g_{[l-k]} + l\ L_{[l]}\ g_{[0]}$$

despejando

$$L_{[l]} = \frac{g_{[l]}}{g_{[0]}} - \frac{1}{l\ g_{[0]}} \sum^{l-1}_{k=0} k\ L_{[k]}\ g_{[l-k]}$$

con las condiciones iniciales 

$$g_{[0]} = g(x_0)$$ y $$L_{[0]} = log(g(x_0))$$

### $P_\alpha(x)$

De manera análoga al caso anterior llegamos a la ecuación

$$\sum^\infty_{n=1} n\ P_{\alpha\ [n]} (x-x_0)^{n-1} = \alpha \Big(\sum_{k=0}^\infty g_{[k]} (x-x_0)^k\Big)^{\alpha-1}
\Big(\sum_{k_0=1}^\infty k_0\ g_{[k_0]} (x-x_0)^{k_0-1}\Big)$$

fijandonos de momento únicamente en el lado derecho de la igualdad notamos que

$$A\equiv\alpha \Big(\sum_{k=0}^\infty g_{[k]} (x-x_0)^k\Big)^{\alpha-1} 
\Big(\sum_{k_0=1}^\infty k_0\ g_{[k_0]} (x-x_0)^{k_0-1}\Big) = 
\alpha \Big(\sum_{k=0}^\infty g_{[k]} (x-x_0)^k\Big)^{\alpha-2} 
\Big(\sum_{k_1=0}^\infty g_{[k_1]} (x-x_0)^{k_1}\Big) 
\Big(\sum_{k_0=1}^\infty k_0\ g_{[k_0]} (x-x_0)^{k_0-1}\Big)$$

y aplicando nuestra regla para multiplicación de sumas en los ultimos ultimos dos terminos obtenemos

$$A = \alpha \Big(\sum_{k=0}^\infty g_{[k]} (x-x_0)^k\Big)^{\alpha-2} 
\sum^\infty_{k_1=1} \Big[\sum^{k_1}_{k_0=0} k_0\ g_{[k_0]}\ g_{[k_1-k_0]}\Big] (x - x_0)^{k_1-1}$$

Si definimos

$$A_{0} \equiv k_0\ g_{[k_{0}]}$$

$$A_{m} \equiv \sum^{k_m}_{k_{m-1}=0}  A_{m-1} g_{[k_m-k_{m-1}]}$$

notamos que en especial

$$A_1 = \sum^{k_1}_{k_{0}=0}  A_0 g_{[k_1-k_{0}]} = \sum^{k_1}_{k_{0}=0} k_0\ g_{[k_{0}]}  g_{[k_1-k_{0}]}$$

entonces

$$A = \alpha \Big(\sum_{k=0}^\infty g_{[k]} (x-x_0)^k\Big)^{\alpha-2} 
\sum^\infty_{k_1=1} A_1 (x - x_0)^{k_1-1}$$

$$A = \alpha \Big(\sum_{k=0}^\infty g_{[k]} (x-x_0)^k\Big)^{\alpha-3} 
\Big(\sum_{k_2=0}^\infty g_{[k_2]} (x-x_0)^{k_2}\Big)
\Big(\sum^\infty_{k_1=1} A_1 (x - x_0)^{k_1-1}\Big)$$

y nuevamente aplicando nuestra regla demultiplicación de sumas a los ultimos dos terminos

$$A = \alpha \Big(\sum_{k=0}^\infty g_{[k]} (x-x_0)^k\Big)^{\alpha-3} 
\sum^\infty_{k_2=1} \Big[\sum^{k_2}_{k_1=0} A_1\ g_{[k_2-k_1]}\Big] (x - x_0)^{k_2-1}$$

$$A = \alpha \Big(\sum_{k=0}^\infty g_{[k]} (x-x_0)^k\Big)^{\alpha-3} 
\sum^\infty_{k_2=1} A_2 (x - x_0)^{k_2-1}$$

repitiendo el mismo procedimiento llegamos a que

$$A = \alpha \sum^\infty_{k_{\alpha-1}=1} A_{\alpha-1} (x - x_0)^{k_{\alpha-1}-1}$$

Por lo que sustituyendo en nuestra primera ecuación

$$\sum^\infty_{n=1} n\ P_{\alpha\ [n]} (x-x_0)^{n-1} = 
\sum^\infty_{k_{\alpha-1}=1} \alpha\ A_{\alpha-1} (x - x_0)^{k_{\alpha-1}-1}$$

Por tanto si $n \neq 0$

$$ P_{\alpha\ [n]} = \frac{\alpha}{n} A_{\alpha -1}$$

con la condición inicial

$$P_{\alpha [0]} = g^{\alpha}(x_0)$$

$$ P_{\alpha\ [n]} = \frac{\alpha}{n} \sum^{\alpha -1}_{k_{m-1}=0}  A_{\alpha-2} g_{[k_{\alpha-1}-k_{m-1}]}$$

In [13]:
import Base: exp, log, sin, cos

In [14]:
function evaluar(a::Taylor,x₀)
    typeof(x₀) <: Number || error("x₀ = '$x₀' debe ser de tipo T<:Number")
    x₀ == 0 && return a.co[1]
    a = Taylor(a.co)
    A = a.co[1]
    for i in 2:a.gr+1
        A += a.co[i] * x₀^(i-1)
    end
    return A
end

evaluar (generic function with 1 method)

In [15]:
function derivada(a::Taylor)
    grado_original = a.gr
    a = Taylor(a.co)
    a_derivada = zeros(typeof(promote(a.co)[1][1]),a.gr+1)
    for i in 2:a.gr+1
        a_derivada[i-1] = a.co[i]*(i-1)
    end
    return Taylor(a_derivada,grado_original)
end

function derivada(a::Taylor,n::Int)
    n < 0 && error("n = $n debe ser mayor o igual a 0")
    n > findlast(a.co)-1 && return Taylor([0],a.gr+1)
    for i in 1:n
        a = derivada(a)
    end
    return a
end

derivada (generic function with 2 methods)

In [27]:
function exp(a::Taylor,x₀=0,n::Int =10)
    typeof(x₀) <: Number || error("x₀ = '$x₀' debe ser de tipo T<:Number")
    E₀ = exp(evaluar(a, x₀)) 
    E = [E₀]
    g(k) = 1/factorial(k)*evaluar(derivada(a,k),x₀)
    for k in 1:n
        push!(E,0)
        for j in 0:k-1
            E[k+1] += 1/(k)*(k-j)*g(k-j)*E[j+1]
        end
    end
    Taylor(E)       
end

exp (generic function with 15 methods)

In [17]:
exp(Taylor([0,1]),0)

Taylor{Float64}([1.0,1.0,0.5,0.16666666666666666,0.041666666666666664,0.008333333333333333,0.0013888888888888887,0.00019841269841269839,2.4801587301587298e-5,2.7557319223985884e-6,2.7557319223985883e-7],10)

In [18]:
for i in 1:10
    a = 1/factorial(i)
    println("$a")
end

1.0
0.5
0.16666666666666666
0.041666666666666664
0.008333333333333333
0.001388888888888889
0.0001984126984126984
2.48015873015873e-5
2.7557319223985893e-6
2.755731922398589e-7


In [25]:
function log(a::Taylor,x₀=0,n::Int =10)
    typeof(x₀) <: Number || error("x₀ = '$x₀' debe ser de tipo T<:Number")
    g₀ = evaluar(a, x₀)
    L₀ = log(g₀)
    g(k) = 1/factorial(k)*evaluar(derivada(a,k),x₀)   
    L = [L₀,g(1)/g₀]
    for k in 2:n
        push!(L,g(k)/g₀)
        for j in 1:k-1
            L[k+1] -= 1/(k*g₀) * j*L[j+1]*g(k-j)
        end
    end
    Taylor(L)       
end

log (generic function with 22 methods)

Por ejemplo:

Si $g(x) = x$ alrededor de $x_0 = 2$, para $k>0$ $g_{[k]} = \delta_{1,k}$ y $g_{[0]} = g(x_0) = 2$ por tanto

$$L[k] = \frac{\delta_{1,k}}{g_{[0]}} - \frac{k-1}{k g_{[0]}} L_{[k-1]}$$

$$L[1] = \frac{1}{g_{[0]}}$$

$$L[2] = - \frac{1}{2 g_{[0]}} L_{[1]}$$

$$L[3] = - \frac{2}{3 g_{[0]}} L_{[2]} = \frac{(2)(1)}{3(2) g_{[0]}^3} = (-1)^{3-1}\frac{(3-1)!}{3! g_{[0]}^3}$$

por lo que para $k>0$

$$L[k] = (-1)^{k-1} \frac{(k-1)!}{k!g_{[0]}^k} = (-1)^{k-1} \frac{1}{kg_{[0]}^k} $$

$$L[k] = (-1)^{k-1} \frac{1}{2^k k}$$

In [20]:
log(Taylor([0,1]),2)

Taylor{Float64}([0.6931471805599453,0.5,-0.125,0.041666666666666664,-0.015625,0.00625,-0.0026041666666666665,0.0011160714285714285,-0.00048828125,0.00021701388888888888,-9.765624999999999e-5],10)

In [21]:
log(2)

0.6931471805599453

In [22]:
for i in 1:10
    a = (-1)^(i-1) * 1/(2^i*i)
    println("$(a)")
end

0.5
-0.125
0.041666666666666664
-0.015625
0.00625
-0.0026041666666666665
0.0011160714285714285
-0.00048828125
0.00021701388888888888
-9.765625e-5


In [23]:
a = Taylor([1,1])*Taylor([1,1])

Taylor{Int64}([1,2,1],2)

In [28]:
exp(a)

Taylor{Float64}([2.718281828459045,5.43656365691809,8.154845485377136,9.060939428196818,8.607892456786978,7.067532753993518,5.225141736926831,3.5121927116915286,2.1843336121545898,1.2658947386324706,0.690045670157412],10)

In [29]:
log(a)

Taylor{Float64}([0.0,2.0,-1.0,0.6666666666666666,-0.5,0.4,-0.3333333333333333,0.2857142857142857,-0.25,0.22222222222222224,-0.2],10)

In [34]:
exp(log(a))

Taylor{Float64}([1.0,2.0,1.0,0.0,0.0,0.0,-5.551115123125783e-17,-1.2688263138573217e-16,-1.7842870038618585e-17,1.1895246692412392e-17,1.9428902930940238e-17],10)

In [45]:
log(exp(a))

Taylor{Float64}([1.0,2.0,1.0,2.220446049250313e-16,3.3306690738754696e-16,-7.105427357601003e-16,1.480297366166874e-16,2.2204460492503185e-16,1.4802973661668258e-17,-1.5049689889363207e-16,-1.0203478273936064e-17],10)

In [44]:
l = Taylor([0,1])
m = Taylor([1])
l/m

Taylor{Float64}(Float64[],0)

In [91]:
function p(α::Int,a::Taylor,x₀,n::Int =10)
    typeof(x₀) <: Number || error("x₀ = '$x₀' debe ser de tipo T<:Number")
    g₀ = evaluar(a, x₀)
    g(k) = 1/factorial(k)*evaluar(derivada(a,k),x₀)  
    P₀ = g₀^α
    function A(n)
        A = zeros(n)
        A[1] = g(1)*g₀ 
        for i in 2:n
            push!(A,A[i-1]*g(i))
        end
        return sum(A)
    end 
    P = [P₀]
    A_α = α*A(α-1)
    for k in 1:n
        s = A_α/k
        push!(P,s)
    end
    return Taylor(P) 
end

p (generic function with 2 methods)

In [92]:
p(2,Taylor([1,1]),0,10)

LoadError: LoadError: InexactError()
while loading In[92], in expression starting on line 1